# Reverse numerical methods applied to channel flow

Our goals here, in general, are to find new valid equations that describe fluid flow. 
We will try random PDEs and use the steady RANS equations to attempt to solve for the Reynolds
stresses in terms of the other quantities we know.


## Steady RANS equations with generic Reynolds stress effects

$$
(\vec{U} \cdot \nabla) \vec{U}
+ \frac{1}{\rho} \nabla P 
- \nu \nabla^2 \vec{U}
= \mathbf{R},
$$

where in this case $\mathbf{R}$ is simply the effects of the Reynolds stresses (i.e., the opposite of the gradient), not the Reynolds stresses themselves.

Now, let's add some terms and assume that their coefficients are isotropic:

$$
(\vec{U} \cdot \nabla) \vec{U}
+ \frac{1}{\rho} \nabla P 
- \nu \nabla^2 \vec{U}
= 
  A (\nabla P)^2
+ B \nabla P^2
+ C (\vec{U} \cdot \nabla) \vec{U}^2
+ D \nabla^2 \vec{U}^2
+ E \nabla K
$$

$$
U \frac{\partial U}{\partial x} 
+ V \frac{\partial U}{\partial y} + W \frac{\partial U}{\partial z}
= - \frac{1}{\rho}\frac{\partial P}{\partial x} + \nu \left( \frac{\partial^2 U}{\partial x^2}
+ \frac{\partial^2 U}{\partial y^2} + \frac{\partial^2 U}{\partial z^2} \right)
+ R_x
$$

$$
U \frac{\partial V}{\partial x} 
+ V \frac{\partial V}{\partial y} + W \frac{\partial V}{\partial z}
= - \frac{1}{\rho}\frac{\partial P}{\partial y} + \nu \left( \frac{\partial^2 V}{\partial x^2}
+ \frac{\partial^2 V}{\partial y^2} + \frac{\partial^2 V}{\partial z^2} \right)
+ R_y
$$

$$
U \frac{\partial W}{\partial x} 
+ V \frac{\partial W}{\partial y} + W \frac{\partial W}{\partial z}
= - \frac{1}{\rho}\frac{\partial P}{\partial z} + \nu \left( \frac{\partial^2 W}{\partial x^2}
+ \frac{\partial^2 W}{\partial y^2} + \frac{\partial^2 W}{\partial z^2} \right)
+ R_z
$$


## Quantities available from JHTDB

* Velocity (`GetVelocity`)
* Pressure (`GetPressure`)
* Velocity gradient (`GetVelocityGradient`)
* Pressure gradient (`GetPressureGradient`)
* Velocity Laplacian (`GetVelocityLaplacian`)
* Velocity Hessian (`GetVelocityHessian`)
* Pressure Hessian (`GetPressureHessian`)


## Algorithm

1. Pick terms (in addition to non-Reynolds stress Navier--Stokes terms).
2. Create a random list of points in space that is at least as large as the number
   of terms.
3. At each point, acquire all data for all terms for all times.
4. Average data at each point for all times.
5. Solve for coefficients using a linear model.

## Terms

$$
U \frac{\partial U}{\partial x} 
+ V \frac{\partial U}{\partial y} + W \frac{\partial U}{\partial z}
+ \frac{1}{\rho}\frac{\partial P}{\partial x} 
- \nu \left( 
    \frac{\partial^2 U}{\partial x^2}
    + \frac{\partial^2 U}{\partial y^2} 
    + \frac{\partial^2 U}{\partial z^2} 
\right)
$$

$$
= 
A \left( \frac{\partial U}{\partial x} \right)^2 
+ B \left( \frac{\partial U}{\partial y} \right)^2
+ C \left( \frac{\partial U}{\partial z} \right)^2
+ D \left( \frac{\partial P}{\partial x} \right)^2
+ E \frac{\partial^2 P}{\partial x^2}
+ F U \frac{\partial P}{\partial x}
$$

$$
U \frac{\partial V}{\partial x} 
+ V \frac{\partial V}{\partial y} 
+ W \frac{\partial V}{\partial z}
+ \frac{1}{\rho}\frac{\partial P}{\partial y} 
- \nu \left( 
    \frac{\partial^2 V}{\partial x^2}
    + \frac{\partial^2 V}{\partial y^2} 
    + \frac{\partial^2 V}{\partial z^2} 
\right)
$$

$$
= 
A \left( \frac{\partial V}{\partial x} \right)^2 
+ B \left( \frac{\partial V}{\partial y} \right)^2
+ C \left( \frac{\partial V}{\partial z} \right)^2
+ D \left( \frac{\partial P}{\partial y} \right)^2
+ E \frac{\partial^2 P}{\partial y^2}
+ F V \frac{\partial P}{\partial y}
$$

In [21]:
import numpy as np
import pandas as pd
import pyJHTDB
from pyJHTDB import libJHTDB
from pyJHTDB.dbinfo import channel as info

info

{'name': 'channel',
 'xnodes': array([0.0000000e+00, 1.2271847e-02, 2.4543693e-02, ..., 2.5095926e+01,
        2.5108198e+01, 2.5120470e+01], dtype=float32),
 'ynodes': array([-1.        , -0.9999835 , -0.999945  , -0.9998791 , -0.99978054,
        -0.9996437 , -0.99946326, -0.9992338 , -0.99896646, -0.9986612 ,
        -0.9983181 , -0.9979371 , -0.99751824, -0.99706155, -0.9965671 ,
        -0.99603474, -0.9954647 , -0.99485683, -0.99421126, -0.99352795,
        -0.99280703, -0.9920484 , -0.9912521 , -0.99041826, -0.98954684,
        -0.98863786, -0.98769146, -0.9867075 , -0.9856862 , -0.9846274 ,
        -0.98353136, -0.982398  , -0.98122734, -0.98001945, -0.9787744 ,
        -0.9774923 , -0.97617304, -0.97481674, -0.97342354, -0.9719934 ,
        -0.97052634, -0.9690225 , -0.9674819 , -0.9659046 , -0.9642907 ,
        -0.96264017, -0.9609531 , -0.95922965, -0.9574698 , -0.95567364,
        -0.9538412 , -0.9519726 , -0.95006794, -0.94812715, -0.9461505 ,
        -0.9441379 , -0.94208

In [22]:
all_times = info["time"]
nu = info["nu"]
all_x = info["xnodes"]
all_y = info["ynodes"]
all_z = info["znodes"]
dataset_name = info["name"]

In [ ]:
db = libJHTDB()
db.initialize()


def get_data_all_time(x, y, z, f="getVelocity", **kwargs):
    """Get data for all times.
    
    interp_info = [(44, 'FD4Lag4'),
                   (104, 'M1Q4'),
                   (208, 'M2Q8'),
                   (214, 'M2Q14')]
    
    """
    vals = []
    for t in all_times:
        vals.append(db.getData(t, np.array((x, y, z)), getFunction=f, **kwargs))
    return np.asarray(vals)


# Pick 100 random points in space
seed = 5
size = 2
np.random.seed(seed)
x = all_x[np.random.randint(0, high=len(all_x) - 1, size=size)]
y = all_y[np.random.randint(0, high=len(all_y) - 1, size=size)]
z = all_z[np.random.randint(0, high=len(all_z) - 1, size=size)]

# Create a DataFrame for all points and all times
all_data = {}
for xi, yi, zi in zip(x, y, z):
    df = pd.DataFrame(index=all_times)
    d = get_data_all_time(xi, yi, zi)
    for n in range(d.shape[1]):
        df["col{}".format(n + 1)] = d[:, n]
    all_data[xi, yi, zi] = df    


db.finalize()

In [18]:
list(all_data.keys())

[(10.63969, 0.8071313, 0.67495155),
 (2.5280004, 0.7653094, 9.062758),
 (24.92412, 0.66676253, 3.35635),
 (21.168936, -0.12592553, 3.0127382),
 (12.247303, -0.1442331, 1.0492429),
 (14.014448, -0.982398, 0.6135923),
 (17.47511, -0.98122734, 4.712389),
 (0.89584476, 0.45392892, 0.79767),
 (12.664545, -0.4919114, 5.510059),
 (9.081166, 0.7326765, 3.0495539),
 (21.684353, 0.76925945, 6.056156),
 (24.3228, -0.47574106, 4.3258257),
 (5.043729, -0.8386031, 4.460816),
 (7.657632, 0.4865397, 1.5646604),
 (8.222137, 0.71137273, 1.5769322),
 (21.4021, 0.98954684, 3.669282),
 (18.93546, -0.67589235, 3.4729326),
 (2.5034566, -0.2528271, 5.645049),
 (23.746023, 0.99978054, 4.3196898),
 (13.95309, -0.9954647, 6.7556515),
 (17.929167, -0.9244226, 5.503923),
 (24.825945, 0.052328087, 1.6260196),
 (13.548119, 0.97481674, 1.9634954),
 (0.33133984, -0.9642907, 2.4973207),
 (13.106332, -0.7156881, 1.0246992),
 (2.5157285, -0.5289798, 5.454836),
 (21.34074, -0.8386031, 2.8532043),
 (13.36404, 0.95006794, 6